In [13]:
import plotly.express as px # might not need this.
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import DBSCAN

from sklearn.cluster import AgglomerativeClustering

In [20]:
df = pd.read_csv(
    '../data/cleaned/final_combined.csv',
    dtype = {'fips':object}
)
df

,fips,total_pop_age_sex,median_age,sex_ratio_males,under_18_percent,over_18_percent,over_65_percent,county,state,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,tot_pop_race_ethnicity,tot_white,tot_black_af_am,tot_aminalnat,tot_asian,tot_hawopi,total_pop_poverty,pop_below_poverty,percent_below_poverty,smoking_ban_2010,asian_2019,avg_family_size_2019,black_2019,hispanic_2019,household_has_broadband_2019,household_has_computer_2019,household_has_smartphone_2019,households_2019,households_speak_limited_english_2019,housing_mobile_homes_2019,hs_grad_2019,median_household_income_2019,median_individual_income_2019,native_2019,other_single_race_2019,pac_isl_2019,persons_per_household_2019,pop_2019,unemployment_rate_2019,uninsured_2019,veterans_2019,white_2019,white_not_hispanic_2019,Metro2013,rural_urban_continuum_code_2013,retirement_destination_2015_update,metro_adjacent_2013,average_2011_upper_bound_pct_obesity,FIPS,obesity_prev_100K
0,01001,55380.0,38.2,94.7,23.8,76.2,15.0,Autauga County,Alabama,21397.0,58731.0,75326.0,15076.0,71103.0,87094.0,55380.0,42527.0,10538.0,140.0,573.0,26.0,54922.0,8340.0,15.2,none,1.0,3.09,19.0,2.8,80.6,73.0,78.4,21397.0,0.7,26.7,88.5,58731.0,29725.0,0.3,0.7,0.0,2.56,55380.0,3.5,7.1,12.6,76.8,74.6,2.0,1.0,1.0,0.0,42.970,1001,71.952925
1,01003,212830.0,43.0,94.7,21.7,78.3,20.0,Baldwin County,Alabama,80930.0,58320.0,80986.0,53467.0,75850.0,97991.0,212830.0,183471.0,19718.0,1645.0,1969.0,9.0,209618.0,21704.0,10.4,none,0.9,3.24,9.3,4.6,81.8,76.3,81.7,80930.0,1.2,24.8,90.8,58320.0,29802.0,0.8,1.1,0.0,2.59,212830.0,4.0,8.9,11.8,86.2,83.1,3.0,1.0,1.0,0.0,37.685,1003,58.781387
2,01005,25361.0,40.4,112.4,20.9,79.1,18.6,Barbour County,Alabama,9345.0,32525.0,47068.0,6187.0,41704.0,56374.0,25361.0,11869.0,12066.0,82.0,134.0,1.0,22417.0,6875.0,30.7,partial,0.5,3.01,47.6,4.4,60.5,51.9,64.2,9345.0,1.6,39.1,73.2,32525.0,17963.0,0.3,3.6,0.0,2.41,25361.0,9.4,11.3,6.6,46.8,45.8,6.0,0.0,0.0,1.0,51.675,1005,63.554734
3,01007,22493.0,40.9,117.5,20.6,79.4,15.9,Bibb County,Alabama,6891.0,47542.0,60182.0,4789.0,57891.0,69316.0,22493.0,17272.0,5014.0,30.0,27.0,0.0,20632.0,3740.0,18.1,none,0.1,3.74,22.3,2.6,69.2,54.7,66.6,6891.0,0.6,25.6,79.1,47542.0,21958.0,0.1,0.0,0.0,2.99,22493.0,7.0,10.7,8.0,76.8,74.5,1.0,1.0,0.0,0.0,47.940,1007,79.901207
4,01009,57681.0,40.7,97.6,23.2,76.8,17.9,Blount County,Alabama,20847.0,49358.0,65639.0,14874.0,62295.0,76547.0,57681.0,55062.0,928.0,46.0,212.0,25.0,57108.0,7739.0,13.6,none,0.4,3.33,1.6,9.3,73.0,63.5,70.1,20847.0,1.8,21.2,80.5,49358.0,26976.0,0.1,0.9,0.0,2.74,57681.0,3.1,10.8,7.7,95.5,86.9,1.0,1.0,0.0,0.0,45.425,1009,65.344687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,56037,43521.0,35.3,106.8,26.5,73.5,11.4,Sweetwater County,Wyoming,15523.0,74843.0,85346.0,10794.0,87906.0,97459.0,43521.0,40642.0,502.0,508.0,340.0,19.0,42958.0,4921.0,11.5,none,0.8,3.32,1.2,15.9,84.0,81.9,83.4,15523.0,1.9,23.8,92.8,74843.0,36512.0,1.2,1.2,0.0,2.77,43521.0,5.7,11.3,8.6,93.4,79.6,5.0,0.0,0.0,0.0,39.170,56037,57.186138
3128,56039,23280.0,39.3,112.5,19.1,80.9,14.0,Teton County,Wyoming,9019.0,84678.0,132531.0,5190.0,110667.0,172024.0,23280.0,20785.0,290.0,77.0,291.0,29.0,23188.0,1426.0,6.1,partial,1.3,3.01,1.2,15.0,87.1,90.1,85.3,9019.0,5.7,39.6,95.2,84678.0,40914.0,0.3,7.1,0.1,2.47,23280.0,0.7,12.7,5.3,89.3,81.3,7.0,0.0,0.0,0.0,23.825,56039,41.840714
3129,56041,20479.0,35.8,103.3,29.2,70.8,13.0,Uinta County,Wyoming,7597.0,63403.0,74938.0,5313.0,77725.0,86265.0,20479.0,19132.0,23.0,145.0,32.0,0.0,20259.0,2298.0,11.3,none,0.2,3.23,0.1,9.1,89.5,80.3,84.8,7597.0,0.6,22.9,92.7,63403.0,29557.0,0.7,2.1,0.0,2.66,20479.0,5.5,11.2,7.4,93.4,87.5,7.0,0.0,0.0,0.0,39.580,56041,53.049534
3130,56043,8027.0,42.9,102.6,24.6,75.4,21.1,Washakie County,Wyoming,3365.0,54158.0,65496.0,2191.0,68265.0,77355

In [21]:
# Smoking ban is the only variable that has null values. We'll check what the prevalent one is and fillna().
df.isna().sum()

fips                                      0
total_pop_age_sex                         0
median_age                                0
sex_ratio_males                           0
under_18_percent                          0
over_18_percent                           0
over_65_percent                           0
county                                    0
state                                     0
total_hh                                  0
median_hh_income                          0
mean_hh_income                            0
total_families                            0
median_family_income                      0
mean_family_income                        0
tot_pop_race_ethnicity                    0
tot_white                                 0
tot_black_af_am                           0
tot_aminalnat                             0
tot_asian                                 0
tot_hawopi                                0
total_pop_poverty                         0
pop_below_poverty               

In [16]:
df['smoking_ban_2010'].value_counts(normalize=True)

none             0.618480
partial          0.203799
comprehensive    0.177721
Name: smoking_ban_2010, dtype: float64

In [17]:
df['smoking_ban_2010'].fillna(0, inplace=True)

In [18]:
df['smoking_ban_2010'].isna().sum()

0

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3132 entries, 0 to 3131
Data columns (total 55 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   fips                                   3132 non-null   object 
 1   total_pop_age_sex                      3132 non-null   float64
 2   median_age                             3132 non-null   float64
 3   sex_ratio_males                        3132 non-null   float64
 4   under_18_percent                       3132 non-null   float64
 5   over_18_percent                        3132 non-null   float64
 6   over_65_percent                        3132 non-null   float64
 7   county                                 3132 non-null   object 
 8   state                                  3132 non-null   object 
 9   total_hh                               3132 non-null   float64
 10  median_hh_income                       3132 non-null   float64
 11  mean

#### Let's try some clustering algorithms.

In [31]:
# Checking to see which of these duplicate columns we should drop. 

df[['state', 
    'county',
    'pop_2019',
    'total_pop_age_sex',
    'white_2019',
    'white_not_hispanic_2019',
    'native_2019',
    'pac_isl_2019',
    'black_2019',
    'hispanic_2019',
    'asian_2019',
    'Metro2013',
    'rural_urban_continuum_code_2013',
    'retirement_destination_2015_update'
   ]]

,state,county,pop_2019,total_pop_age_sex,white_2019,white_not_hispanic_2019,native_2019,pac_isl_2019,black_2019,hispanic_2019,asian_2019,Metro2013,rural_urban_continuum_code_2013,retirement_destination_2015_update
0,Alabama,Autauga County,55380.0,55380.0,76.8,74.6,0.3,0.0,19.0,2.8,1.0,2.0,1.0,1.0
1,Alabama,Baldwin County,212830.0,212830.0,86.2,83.1,0.8,0.0,9.3,4.6,0.9,3.0,1.0,1.0
2,Alabama,Barbour County,25361.0,25361.0,46.8,45.8,0.3,0.0,47.6,4.4,0.5,6.0,0.0,0.0
3,Alabama,Bibb County,22493.0,22493.0,76.8,74.5,0.1,0.0,22.3,2.6,0.1,1.0,1.0,0.0
4,Alabama,Blount County,57681.0,57681.0,95.5,86.9,0.1,0.0,1.6,9.3,0.4,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,Wyoming,Sweetwater County,43521.0,43521.0,93.4,79.6,1.2,0.0,1.2,15.9,0.8,5.0,0.0,0.0
3128,Wyoming,Teton County,23280.0,23280.0,89.3,81.3,0.3,0.1,1.2,15.0,1.3,7.0,0.0,0.0
3129,Wyoming,Uinta County,20479.0,20479.0,93.4,87.5,0.7,0.0,0.1,9.1,0.2,7.0,0.0,0.0
3130,Wyoming,Washakie County,8027.0,8027.0,89.7,81.9,0.9,0.0,0.0,14.2,0.0,7.0,0.0,0.0


In [ ]:
X = df.drop(columns = ['tot_pop_race_ethnicity',
                      'total_pop_poverty',
                      'median_household_income_2019',
                      'median_individual_income_2019',
                       'tot_white',
                       'tot_black_af_am',
                       'tot_aminalnat',
                       'tot_asian',
                       'tot_hawopi',
                       'white_2019',
                       'pop_below_poverty',
                       'under_18_percent',
                       'mean_hh_income',
                       'total_families',
                       'mean_family_income',
                       'total_pop_age_sex'
                     ])